In [1]:
from Signal_generator.generate_signal import generate_X_matrix
from Algorithmes.beamforming import beamforming_method
from Signal_generator.generate_signal import generate_A_matrix
from Signal_generator.generate_signal import generate_S_matrix
from Signal_generator.generate_signal import generate_noise
from Signal_generator.generate_signal import generate_R_hat
from Signal_generator.generate_signal import generate_R_hat_with_phase
from Algorithmes.music import music_method
from Algorithmes.music import estimate_angles
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split 
import numpy as np
import torch
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import itertools
from torchsummary import summary
from numpy.linalg import eigh
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchsummary import summary
import math
import random
from sklearn.metrics import mean_squared_error
from Algorithmes.music import generate_steering_vector
from Plots.draw_plot import plot_single_music

In [2]:
nbSources = 1 # Nombre de sources  dans l'article ils fixent le nombre de sources à 2
nbSensors = 9 # Nombre de capteurs
nbTimePoints = 100 # Nombre de points temporels


#Pour l'échantillon d'entraînements ils utilisent une observation de chaque combinaison d'angles d'arrivée possible pour chaque SNR level, dans le cas d'une seule source, 2G+1 observations
#SNR ratio dans le papier est seulement low, de -20 à 0 DB avec un pas de 5
#Petite note : si on fait un modèle pour chaque SNR level, les performances augmentent légérements, mais cela ne vaut pas l'effort effectué
signal_noise_ratio = 3 # Rapport signal sur bruit en décibels. Si 'False', cela revient à une absence totale de bruit.
L = nbSensors
T = 400
correlation_List = [] # Liste des corrélations. Il y a une corrélation nécéssaire pour chaque paire distincte de sources différentes: 0 pour 1 source, 1 pour 2 sources, 3 pour 3 sources, 6 pour 4 sources etc...
# Ordre de remplisage de la correlation_List: de gauche à droite et ligne par ligne, moitié haut-droite de la matrice uniquement, puis symétrie de ces valeurs pour la moitié bas-gauche.
SNR_TRAIN = 30
Q = 10
N = 2**12 
theta_start = -45
theta_final = 45


In [3]:
import numpy as np

def calculate_music_spectrum(R_hat, nbSensors, nbSources, theta_range):
    eigenvalues, eigenvectors = np.linalg.eigh(R_hat)
    noise_subspace = eigenvectors[:, :nbSensors - nbSources]
    music_spectrum = np.zeros(len(theta_range))

    for i, theta in enumerate(theta_range):
        steering_vector = generate_steering_vector(nbSensors, theta)
        music_spectrum[i] = 1 / np.abs(np.conj(steering_vector).T @ noise_subspace @ noise_subspace.conj().T @ steering_vector)

    return music_spectrum



In [4]:
def generate_deepmusic_partitioned_data(nbSensors, nbSources, T, J_alpha, J_beta, SNR_TRAIN, Q, N, theta_start, theta_final):
    training_data_sets = [[] for _ in range(Q)]

    for j_alpha in range(J_alpha):
        thetaList = np.random.uniform(theta_start, theta_final, nbSources).tolist()

        for j_beta in range(J_beta):
            varList = np.ones(nbSources).tolist()
            correlation_List = np.zeros(nbSources * (nbSources - 1) // 2).tolist()
            X = generate_X_matrix(nbSources, nbSensors, T, thetaList, varList, correlation_List, SNR_TRAIN)
            
            R_hat = generate_R_hat(X)
            full_theta = np.linspace(theta_start, theta_final, N)
            full_music_spectrum = calculate_music_spectrum(R_hat, nbSensors, nbSources, full_theta)

            for q in range(Q):
                # Sampling MUSIC spectrum for the q-th subregion
                start_index = q * L
                end_index = start_index + L
                pq = full_music_spectrum[start_index:end_index]
                
                input_data = generate_R_hat_with_phase(X)
                output_data = pq
                training_data_sets[q].append((input_data, output_data))                

    return training_data_sets

# Example usage
J_alpha = 100  # Number of DOA angle sets to generate
J_beta = 10   # Number of iterations for each set of DOA angles
SNR_TRAIN_list = [15,20, 25, 30] # Different SNR levels for training
theta_range = np.linspace(theta_start, theta_final, N)
training_datasets = generate_deepmusic_partitioned_data(nbSensors, nbSources, T, J_alpha, J_beta, SNR_TRAIN, Q, N,theta_start, theta_final)


[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

class DeepMusicDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_data, output_data = self.data[idx]
        return torch.tensor(input_data, dtype=torch.float32), torch.tensor(output_data, dtype=torch.float32)

def create_loaders_for_subregion(data, batch_size=64, validation_split=0.2, test_split=0.2):
    # Séparation des données en ensembles d'entraînement, de validation et de test
    train_data, test_data = train_test_split(data, test_size=test_split, random_state=42)
    train_data, val_data = train_test_split(train_data, test_size=validation_split, random_state=42)

    # Création des instances de DeepMusicDataset
    train_dataset = DeepMusicDataset(train_data)
    val_dataset = DeepMusicDataset(val_data)
    test_dataset = DeepMusicDataset(test_data)

    # Création des DataLoader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

# Création des DataLoader pour chaque sous-région
loaders_per_subregion = [create_loaders_for_subregion(training_datasets[q]) for q in range(Q)]

In [6]:

class DeepMusicModel(nn.Module):
    def __init__(self):
        super(DeepMusicModel, self).__init__()
        # Define the first convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=256, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(num_features=256)
        
        # Define the second convolutional layer
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm2d(num_features=256)
        
        # Define the third convolutional layer
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=256)
        
        # Define the fourth convolutional layer
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(num_features=256)
        
        # Assuming the spatial dimensions (height and width) are reduced to 1x1 after the convolutions
        # Define the fully connected layers
        self.fc1 = nn.Linear(in_features=20736, out_features=9)
        
        # Dropout layer
        self.dropout = nn.Dropout(p=0.5)

        # Softmax layer 
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # Apply the first convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn1(self.conv1(x)))
        
        # Apply the second convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn2(self.conv2(x)))
        
        # Apply the third convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn3(self.conv3(x)))
        
        # Apply the fourth convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn4(self.conv4(x)))
        
        # Flatten the tensor for the fully connected layer
        x = torch.flatten(x, 1)
        
        # Apply the first fully connected layer
        x = self.fc1(x)
        
        # Apply the dropout layer
        x = self.dropout(x)

        #Apply the softmax layer
        x = self.softmax(x)
        
        return x

model = DeepMusicModel()
model

DeepMusicModel(
  (conv1): Conv2d(3, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=20736, out_features=9, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (softmax): Softmax(dim=1)
)

In [7]:
#Définition du device

device = (
    "gpu"
    if torch.cuda.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using cpu device


In [10]:
import torch
import numpy as np
from scipy.signal import find_peaks
import torch.nn.functional as F

def calculate_accuracy(predicted_spectrum, target_spectrum, angles_range, nbSources, peak_tolerance):
    """
    Calculate accuracy for a single predicted and target spectrum.
    
    :param predicted_spectrum: Predicted MUSIC spectrum (1D NumPy array).
    :param target_spectrum: True MUSIC spectrum (1D NumPy array).
    :param angles_range: The range of angles over which the MUSIC spectrum is calculated.
    :param nbSources: The number of sources (peaks) to consider.
    :param peak_tolerance: The tolerance for considering a predicted peak to match a target peak.
    :return: Accuracy as a float.
    """
    # Estimate angles from the predicted and target spectra
    estimated_angles = estimate_angles(nbSources, predicted_spectrum, angles_range)
    true_angles = estimate_angles(nbSources, target_spectrum, angles_range)

    # Compare the estimated angles with the true angles
    correct_predictions = sum(1 for est_angle in estimated_angles if np.any(np.abs(true_angles - est_angle) <= peak_tolerance))
    accuracy = correct_predictions / nbSources if nbSources > 0 else 0
    return accuracy



def train_and_evaluate_spectrum(loaders_per_subregion, epochs, nbSources, angles_range, peak_tolerance=5):
    models = []
    train_loss_list = []
    val_loss_list = []
    val_accuracy_list = []

    # Training configuration
    initial_lr = 0.01
    lr_decay_factor = 0.5
    lr_decay_epoch = 10
    early_stopping_patience = 3

    for train_loader, val_loader, test_loader in loaders_per_subregion:
        model = DeepMusicModel()
        optimizer = torch.optim.SGD(model.parameters(), lr=initial_lr, momentum=0.9)
        criterion = torch.nn.MSELoss()

        best_val_accuracy = 0
        epochs_no_improve = 0

        for epoch in range(epochs):
            # Adjust learning rate
            if epoch % lr_decay_epoch == 0 and epoch > 0:
                for param_group in optimizer.param_groups:
                    param_group['lr'] *= lr_decay_factor

            # Training loop
            model.train()
            train_loss = 0
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
                train_loss += loss

            # Validation loop
            model.eval()
            val_loss, val_accuracy = 0, 0
            with torch.no_grad():
                for inputs, targets in val_loader:
                    outputs = model(inputs)
                    val_loss += criterion(outputs, targets).item()
                    val_accuracy += calculate_accuracy(outputs, targets, angles_range, nbSources, peak_tolerance)

            avg_val_loss = val_loss / len(val_loader)
            avg_train_loss = train_loss / len(train_loader)
            avg_val_accuracy = val_accuracy / len(val_loader)
            val_loss_list.append(avg_val_loss)
            val_accuracy_list.append(avg_val_accuracy)

            # Early stopping
            if avg_val_accuracy > best_val_accuracy:
                best_val_accuracy = avg_val_accuracy
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
                if epochs_no_improve == early_stopping_patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break

        models.append(model)

    # Test set evaluation
    total_mse = 0
    total_peak_similarity = 0
    total_angle_accuracy = 0
    total_batches = 0

    for model, test_loader in zip(models, [x[2] for x in loaders_per_subregion]):
        model.eval()
        with torch.no_grad():
            for inputs, targets in test_loader:
                outputs = model(inputs)

                # Compute MSE for each batch
                mse = F.mse_loss(outputs, targets)
                total_mse += mse.item()

                # Process each item in the batch for peaks and angle estimation
                for output, target in zip(outputs, targets):
                    # Angle accuracy
                    estimated_angles = estimate_angles(nbSources, output.numpy().flatten(), angles_range)
                    angle_accuracy = calculate_accuracy(output.numpy().flatten(), target.numpy().flatten(), angles_range, nbSources, peak_tolerance)
                    total_angle_accuracy += angle_accuracy

                total_batches += 1

    avg_mse = total_mse / total_batches
    avg_angle_accuracy = total_angle_accuracy / total_batches

    print(f'Average Training Loss: {np.mean(train_loss_list):.4f}')
    print(f'Average Validation Loss: {np.mean(val_loss_list):.4f}')
    print(f'Average MSE on Test Set: {avg_mse:.4f}')
    print(f'Average Angle Accuracy on Test Set: {avg_angle_accuracy:.4f}')

# Example usage
train_and_evaluate_spectrum(loaders_per_subregion=loaders_per_subregion, epochs=1000, nbSources=nbSources, angles_range=theta_range, peak_tolerance=5)

Early stopping at epoch 4
Early stopping at epoch 4
Early stopping at epoch 4
Early stopping at epoch 4
Early stopping at epoch 4
Early stopping at epoch 4
Early stopping at epoch 4
Early stopping at epoch 4
Early stopping at epoch 4
Early stopping at epoch 4
Average Training Loss: nan
Average Validation Loss: 348623.9398
Average MSE on Test Set: 186263.1207
Average Angle Accuracy on Test Set: 0.2250
